In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import plotly.express as px

In [ ]:
%run ../../modules/utils.ipynb
%run ../../modules/cds.ipynb
%run ../../modules/preprocessing.ipynb

In [ ]:
fmock = '/Users/xyuan/Documents/BioPharmaFinder/BaoLab/FirstBatch/Excels/S2to5combined_20ul_40min.xlsx'
frsv = '/Users/xyuan/Documents/BioPharmaFinder/BaoLab/FirstBatch/Excels/S6to10combined_20ul_40min.xlsx'
df_mock_ad = pd.read_excel(fmock)
df_rsv_ad = pd.read_excel(frsv)
df_mock_ad = thermo_df(df_mock_ad)
df_rsv_ad = thermo_df(df_rsv_ad)
df_mock_ad.shape, df_rsv_ad.shape

In [ ]:
# bcr_base = base_calling_random(df_mock_ad)
# bcr_2me = base_calling_random(df_mock_ad, methyl_double=True)
bcr_base = base_calling_random(df_rsv_ad)
bcr_2me = base_calling_random(df_rsv_ad, methyl_double=True)
print(bcr_base[0].shape, bcr_2me[0].shape)
# plotly_basecalling(*bcr, annotate=False)

In [ ]:
df_comm = match_dfs(bcr_2me[0], bcr_base[0])
df_comm.shape

In [ ]:
graph = generate_basecalling_graphs(df_rsv_ad)
# graph = generate_basecalling_graphs(df_mock_ad)
len(graph.nodes())

In [ ]:
G = nx.DiGraph()
G_core = nx.DiGraph()
idx = 0
for node in graph.nodes():
#     idx += 1
#     if idx > 10:
#         break
    edge_names = [graph[edge[0]][edge[1]]['name'] for edge in graph.out_edges(node)]
    edge_names = set(edge_names)
#     print(edge_names)
    contains_A = {'A', 'a'}.issubset(edge_names)
    contains_C = {'C', 'c'}.issubset(edge_names)
    contains_G = {'G', 'g'}.issubset(edge_names)
    contains_U = {'U', 'u'}.issubset(edge_names)
    contains_D = {'U', 'D'}.issubset(edge_names)
    if any([contains_A, contains_C, contains_G, contains_U, contains_D]):
        G_core.add_node(node)
        G.add_edges_from(graph.out_edges(node))
        if node < 1500:
            print(node, edge_names)
len(G.nodes()), len(G_core.nodes())

In [ ]:
masses_core = list(G_core.nodes())
masses = list(graph.nodes())
df_core = df_rsv_ad[df_rsv_ad.Mass.isin(masses_core)]
df = df_rsv_ad[df_rsv_ad.Mass.isin(masses)]
# df_tmp = df_mock_ad[df_mock_ad.Mass.isin(masses)]
plotly_zones(df, df_core)
# plotly_zone(df_core)
df_core.shape, df.shape, bcr_base[0].shape

In [ ]:

def paired_ladders(df, G, G_start):
    def direct_nodes(G, node):
        edges = G.out_edges(node)
        edges_names = [G[edge[0]][edge[1]]['name'] for edge in edges]
        print(edges_names)
        nodes = G.successors(node)
        df_nodes = pd.DataFrame({'Mass': nodes})
        df_a = match_dfs(df_nodes, df_nodes, shift=M)
        df_b = match_dfs(df_nodes, df_nodes, shift=-1*M)
        if any([df_a.empty, df_b.empty]):
            return None
        return (df_a.iloc[0].Mass, df_b.iloc[0].Mass)
    
    def next_nodes(G, nodes):
        edges_a = G.out_edges(nodes[0])
        edges_b = G.out_edges(nodes[1])
        edges_a_names = [(edge[0], edge[1], G[edge[0]][edge[1]]['name']) for edge in edges_a]
        edges_b_names = [(edge[0], edge[1], G[edge[0]][edge[1]]['name']) for edge in edges_b]
        comm_name = set([x[2] for x in edges_a_names]) & set([x[2] for x in edges_b_names])
        comm_name = comm_name.pop() if comm_name else None
        if not comm_name:
            return None
        next_node_a = [x[1] for x in edges_a_names if x[2] == comm_name][0]
        next_node_b = [x[1] for x in edges_b_names if x[2] == comm_name][0]
        return (next_node_a, next_node_b)
        
    ladders = list()   
    for node in G_start.nodes():
        ladder_a = list()
        ladder_b = list()
        ladder_a.append(node)
        ladder_b.append(node)
        nodes = direct_nodes(G, node)
        print(nodes)
        while nodes:
            ladder_a.append(nodes[0])
            ladder_b.append(nodes[1])
            nodes = next_nodes(G, nodes)
            if nodes:
                ladder_a.append(nodes[0])
                ladder_b.append(nodes[1])
        ladders.append((ladder_a, ladder_b))
    return ladders

ladder_pairs = paired_ladders(df, graph, G_core)
len(ladder_pairs)

In [ ]:
ladder_pairs.sort(key=lambda x: len(x[0]), reverse=True)
def ladder_pair_df(df, ladder_pair):
    df_a = df_rsv_ad[df_rsv_ad.Mass.isin(ladder_pair[0])]
    df_b = df_rsv_ad[df_rsv_ad.Mass.isin(ladder_pair[1])]
    return (df_a, df_b)
#     plotly_multi_zones([df, df_a, df_b])
    print('ladder length: ', len(ladder_pair[0]))


In [ ]:
pairs = [x for x in ladder_pairs if len(x[0])>=13]

In [ ]:
pairs

In [ ]:
pairs[0]

In [ ]:
df_pair = ladder_pair_df(df, ladder_pairs[0])
# plotly_zones(*df_pair)
# bcr = base_calling_random(df_pair[0])
bcr = base_calling_random(pd.concat(df_pair))
plotly_basecalling(*bcr)

In [ ]:
def generate_basecalling_graphs(df):
    """ given dots, doing basecalling and return a graph contains the connections
    """
    def create_graph(bcr):
        G = nx.DiGraph()
        for x in bcr[1]:
            l = min(x[0], x[1])
            h = max(x[0], x[1])
            G.add_edge(l, h, name=x[2])

        return G
    
    bcr = base_calling_random(df)
    graphs = create_graph(bcr)
    
    return graphs

In [ ]:
def base_calling_random(df, silence=False, methyl=False, methyl_double=False):
    df = df.sort_index()
    mass_x = np.array(df.Mass)
    mass_y = np.array(df.Mass)
    if not silence:
        print("mass_x {} mass_y {}".format(mass_x.shape, mass_y.shape))

#     base_dict = {'C': 305.0413, 'A': 329.0525, 'G': 345.0474, 'U': 306.0253, 'g': 359.0631, 'D': 308.041, 'c': 319.0569, 'u': 320.041, 'G^': 373.0787, 'a': 343.0682, 'y': 212.0086, 'Y': 570.1475, 'P': 557.2251, 'x': 688.1156, 'z': 625.0844, 'I': 330.03654, 'O': 344.052}
#     base_dict = {'C': 305.0413, 'A': 329.0525, 'G': 345.0474, 'U': 306.0253, 'g': 359.0631, 'D': 308.041, 'c': 319.0569, 'u': 320.041, 'a': 343.0682, 'm22C': 333.0625} #, 'y': 212.0086, 'mnm5s2U': 365.04466, 'X': 449.08299, 's4U': 322.00246, 'ms2io6A': 459.09776
#     base_dict = {'g': 359.0631, 'u': 320.041, 'mnm5s2U': 365.04466, 'X': 449.08299, 's4U': 322.00246, 'ms2io6A': 459.09776} #387.1273+61.9557
    base_dict = {'C': 305.0413, 'A': 329.0525, 'G': 345.0474, 'U': 306.0253, 
                 'g': 359.0631, 'c': 319.0569, 'u': 320.041, 'a': 343.0682, 'D': 308.041}
    if methyl:
        base_dict = {'Me': 14.0106} #, '2Me': 28.0212
    if methyl_double:
        base_dict = {'f5C': 333.0362, 'A': 357.0737, 'm22G': 373.0787, 'm5Um': 334.0566}
    pairs = list()
    idxs = list()
    PPM = 10
    for k in base_dict.keys():
        base_mass = base_dict.get(k)
        ppm_matrix = np.abs((mass_x[:, np.newaxis] - mass_y - base_mass) * 1E6 / (mass_y + base_mass))
        ppm_df = pd.DataFrame(ppm_matrix)
        crosstalk = ppm_df[ppm_df < PPM]
        idx_pairs = list(crosstalk[crosstalk.notnull()].stack().index)
        if idx_pairs:
            idx_pairs = [(*pair, k) for pair in idx_pairs] # append base name into idx_pairs
            pairs.extend(idx_pairs)
        df3_idxs = [pair[0] for pair in idx_pairs]
        df5_idxs = [pair[1] for pair in idx_pairs]
        df3_idxs = list(set(df3_idxs))
        df5_idxs = list(set(df5_idxs))
#         if not silence:
#             print("df3_idxs {} df5_idxs {}".format(len(df3_idxs), len(df5_idxs)))
        if df3_idxs:
            idxs.extend(df3_idxs)
        if df5_idxs:
            idxs.extend(df5_idxs)
    
    mass_pairs = [(df.Mass.iloc[p[0]], df.Mass.iloc[p[1]], p[2]) for p in pairs]
    mass_pairs = [mp for mp in mass_pairs if mp[0] != mp[1]] # remove item which contains duplicated values
    idxs = list(set(idxs))
#     plt.figure(figsize=(16, 12))
    df_base_calling = df.iloc[idxs]
    return df_base_calling, mass_pairs